In [5]:
import json
import sys
import re

# function reading the json file 

In [6]:
def read_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
   
def save_json_to_file(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)

In [16]:
# Function to preprocess JSON data
def preprocess_json(data):
    # Fix unescaped backslashes
    data = re.sub(r'(?<!\\)\\\\', '\\\\', data)  # Correctly escape backslashes

    # Further custom processing to fix common errors
    return data
def load_and_clean_json(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Assuming the second line contains JSON
    json_data = lines[1].strip()

    # Preprocess the JSON data before loading
    json_data = preprocess_json(json_data)

    try:
        # Attempt to parse the JSON data
        data = json.loads(json_data)
        print("JSON loaded successfully.")
        return data
    except json.decoder.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        print("Error occurred at position:", e.pos)
        # Optionally print a snippet of the data around the error
        print("Data snippet around error:")
        print(json_data[e.pos - 50:e.pos + 50])  # Print a portion around the error position
        return None
        
json2 = load_and_clean_json('database_newend.txt')
json1 = load_and_clean_json('database_newstart.txt')

JSON loaded successfully.
JSON loaded successfully.


# removing the Keys 

In [14]:
def filter_json_remove_slash_key_value(data, pattern="/key"):
    """
    Recursively removes keys from a dictionary if they contain the given pattern.

    """
    if not isinstance(data, dict):
        return data  # If data is not a dictionary, return as is
    
    new_dict = {}
    for k, v in data.items():
        if pattern not in k:  # Exclude keys that contain the pattern
            if isinstance(v, dict):
                new_dict[k] = filter_json_remove_slash_key_value(v, pattern)  # Recurse for nested dict
            else:
                new_dict[k] = v  # Keep the value as is
    
    return new_dict


In [18]:
save_json_to_file(json2, '24250227_135126_end_original.json')
save_json_to_file(json1, '24250227_135126_start_original.json')
json2=filter_json_remove_slash_key_value(json2)
json1=filter_json_remove_slash_key_value(json1)
save_json_to_file(json2, '24250227_135126_end_remove_slash_key_value.json')
save_json_to_file(json1, '24250227_135126_start_remove_slash_key_value.json')

In [3]:
# def filter_json_remove_slash_key(data, pattern="/key"):
#     # Create a new dictionary to store the filtered results
#     new_dict = {}
    
#     for k, v in data.items():
#         # If the key does not contain the pattern, add it to the new dictionary
#         if pattern not in k:
#             if isinstance(v, dict):
#                 # Recursively apply the function if the value is a dictionary
#                 new_dict[k] = filter_json_remove_slash_key(v, pattern)
#             else:
#                 # Otherwise, just add the key-value pair
#                 new_dict[k] = v
    
#     return new_dict


In [20]:
def filter_json_remove_system_keys(data):
    # Ensure we only modify the 'System' key if it exists
    if 'System' in data:
        # List of keys to remove from 'System'
        keys_to_remove = ['Buffers', 'Transition', 'Flush', 'Tmp']
        
        # Loop through the list and remove each key from 'System' if it exists
        for key in keys_to_remove:
            data['System'].pop(key, None)
    # Filter out entries that have "last_written" in the key
    if isinstance(data, list):  # Check if data is a list of dictionaries
        data = [entry for entry in data if "/last_written" not in entry.get("key", "")]

    return data

In [21]:
def filter_json_remove_legacytrigger(data):
    """
    This function recursively removes keys that match the pattern "/Detectors/DetXX/Settings/LegacyTrigger".
    """
    def recursive_trim(d):
        if isinstance(d, dict):
            # Identify keys matching the specific pattern
            keys_to_remove = [key for key in d if "/Detectors/Det" in key and "/Settings/LegacyTrigger" in key]
            # Remove the matching keys
            for key in keys_to_remove:
                del d[key]
            # Recursively process nested dictionaries or lists
            for key, value in d.items():
                if isinstance(value, (dict, list)):  # If the value is a dictionary or list, apply recursion
                    recursive_trim(value)

    # Call the recursive trim function on the data
    recursive_trim(data)
    
    return data


In [82]:
# Removing last_readout duplicate time 

In [25]:
# def filter_data_remove_duplicate_last_written1(data):
#     seen_values = set()  # Stores unique "last_written" values

#     def recurse(d):
#         if isinstance(d, dict):
#             new_dict = {}
#             for k, v in d.items():
#                 if isinstance(v, dict) and "last_written" in v:
#                     last_written_val = v["last_written"]
#                     if last_written_val not in seen_values:
#                         seen_values.add(last_written_val)
#                         new_dict[k] = v  # Keep the first occurrence
#                 else:
#                     # Recursively process nested dictionaries and lists
#                     new_dict[k] = recurse(v)
#             return new_dict
#         elif isinstance(d, list):
#             return [recurse(item) for item in d]
#         return d

#     # First pass to collect duplicates
#     filtered_data = recurse(data)

#     return filtered_data


In [ ]:
# json2 = read_json_from_file('savedata.json')

# json2 = filter_json_remove_slash_key(json2)


# json2 = filter_json_remove_system_keys(json2)


# json2 = filter_json_remove_legacytrigger(json2)


# json2 = json_filter_remove_last_written(json2)
# save_json_to_file(json2, "s")

In [22]:
# # Function to recursively remove keys ending with "last_written"
# def json_filter_remove_last_written(data):
#     if isinstance(data, dict):
#         return {k: json_filter_remove_last_written(v) for k, v in data.items() if not k.endswith("last_written")}
#     elif isinstance(data, list):
#         return [json_filter_remove_last_written(item) for item in data]
#     return data

In [23]:
import re

def filter_json_differ_Readback_Setting(d):
    """
    Filters out keys from the 'Settings' section of Detectors/DetXX if they match the values in the 'Readback' section.
    Keeps the keys that differ between the 'Settings' and 'Readback' paths.
    """
    def recurse(d, readback_data, settings_data, path=""):
        """
        Recursive function to traverse and compare the two paths.
        - d: the current data (settings or readback)
        - readback_data: the current data from the Readback section
        - settings_data: the current data from the Settings section
        - path: the current path to compare (for debugging or tracking)
        """
        if isinstance(d, dict):
            # Iterate through the dictionary
            new_dict = {}
            for k, v in d.items():
                new_path = f"{path}/{k}" if path else k
                
                # Check if the path is within the Readback or Settings section
                if re.match(r"/Detectors/Det\d+/Readback", new_path):
                    # If the path starts with Readback, compare values from the 'Settings' section
                    if new_path in settings_data and settings_data[new_path] == v:
                        # If values are the same, remove them from the Settings data
                        settings_data.pop(new_path, None)
                    else:
                        new_dict[k] = recurse(v, readback_data, settings_data, new_path)
                elif re.match(r"/Detectors/Det\d+/Settings", new_path):
                    # Handle the Settings path, keep values only if different from Readback
                    if new_path in readback_data and readback_data.get(new_path) != v:
                        new_dict[k] = v
                    else:
                        # Remove from settings if matched with readback
                        settings_data.pop(new_path, None)
                else:
                    new_dict[k] = recurse(v, readback_data, settings_data, new_path)
            return new_dict
        elif isinstance(d, list):
            return [recurse(item, readback_data, settings_data, f"{path}[{idx}]") for idx, item in enumerate(d)]
        return d

    # Extract readback and settings data from the main structure
    readback_data = d.get('Detectors', {}).get('DetXX', {}).get('Readback', {})
    settings_data = d.get('Detectors', {}).get('DetXX', {}).get('Settings', {})

    # Filter out the matching values and keep differing ones
    filtered_data = recurse(d, readback_data, settings_data)
    #print(filtered_data)
    return filtered_data


In [17]:
# def find_differences(start_obj, end_obj, parent_key='', differences=None):
#     if differences is None:
#         differences = {}

#     # Get all the unique keys from both objects
#     all_keys = set(start_obj.keys()).union(set(end_obj.keys()))

#     # Iterate through all keys
#     for key in all_keys:
#         # Build the current key's path
#         current_key = parent_key + key

#         # Case where the values are lists
#         if isinstance(start_obj.get(key), list) or isinstance(end_obj.get(key), list):
#             # Compare lists by joining them into a string for easy comparison
#             if str(start_obj.get(key)) != str(end_obj.get(key)):
#                 differences[current_key] = end_obj.get(key)

#         # Case where the values are dictionaries
#         elif isinstance(start_obj.get(key), dict) and isinstance(end_obj.get(key), dict):
#             # Recurse into the nested dictionaries with updated parent_key
#             find_differences(start_obj.get(key), end_obj.get(key), current_key + '/', differences)

#         # Case where the values are primitive types (strings, numbers, etc.)
#         elif start_obj.get(key) != end_obj.get(key):
#             differences[current_key] = end_obj.get(key)

#     return differences

In [24]:
def find_differences(start_obj, end_obj):
    def recursive_diff(start, end):
        diff = {}
        for key in end.keys():
            if key not in start:
                # Key is new in end_obj
                diff[key] = end[key]
            elif isinstance(start[key], dict) and isinstance(end[key], dict):
                # Recursively check nested dictionaries
                nested_diff = recursive_diff(start[key], end[key])
                if nested_diff:  # Only add if there are differences
                    diff[key] = nested_diff
            elif isinstance(start[key], list) and isinstance(end[key], list):
                # Compare lists directly
                if start[key] != end[key]:
                    diff[key] = end[key]
            elif start[key] != end[key]:
                # Value has changed
                diff[key] = end[key]
        return diff

    return recursive_diff(start_obj, end_obj)
# differences = find_differences(start_json, end_json)
# print(differences)


In [28]:
json2 = read_json_from_file('24250227_135126_end_original.json')
json2 = filter_json_remove_slash_key_value(json2)
json2 = filter_json_remove_system_keys(json2)
json2 = filter_json_remove_legacytrigger(json2)
# json2 = filter_json_differ_Readback_Setting(json2)
json1 = read_json_from_file('24250227_135126_start_original.json')
json1 = filter_json_remove_slash_key_value(json1)
json1 = filter_json_remove_system_keys(json1)
# json1 = filter_json_differ_Readback_Setting(json1)
finddifference=find_differences(json2,json1)

In [36]:
json2 = read_json_from_file('24250227_135126_end_original.json')
json2 = filter_json_remove_slash_key_value(json2)
json2 = filter_json_remove_system_keys(json2)
json2 = filter_json_remove_legacytrigger(json2)
# json2 = filter_json_differ_Readback_Setting(json2)
json1 = read_json_from_file('24250227_135126_start_original.json')
json1 = filter_json_remove_slash_key_value(json1)
json1 = filter_json_remove_system_keys(json1)
finddifference=find_differences(json1,json2)
finddifference

{'Logger': {'Channels': {'0': {'Settings': {'Current filename': '.RUN00075_DUMP0001.mid.gz'},
    'Statistics': {'Disk level': 0.001500701734860943,
     'Bytes written': 4489610,
     'Files written': 20818,
     'Events written': 361,
     'Bytes written total': 17319883542959.0,
     'Bytes written subrun': 2244248,
     'Bytes written uncompressed': 69735101}}}},
 'System': {'Clients': {'2049309': {'Run state': 1},
   '2049311': {'Run state': 1}}},
 'Runinfo': {'Stop time': 'Thu Feb 27 13:53:03 2025',
  'Stop time binary': '0x67c0b48f',
  'Transition in progress': 0},
 'Equipment': {'EBlvl': {'Statistics': {'Events sent': 56}},
  'EBuilder': {'Statistics': {'Events sent': 355},
   'Special dump numbers': {'Last BORR': 1, 'Last BORTS': 1}},
  'L2Trigger': {'Statistics': {'Events sent': 86}},
  'readoutfe_SDU': {'Statistics': {'Events sent': 0}},
  'readoutveto03': {'Statistics': {'Events sent': 10}},
  'L2TriggerHistory': {'Statistics': {'Events sent': 10}},
  'readouthistory03': {'

In [29]:
finddifference

{'Logger': {'Channels': {'0': {'Settings': {'Current filename': '.RUN00075_DUMP0000.mid.gz'},
    'Statistics': {'Disk level': 0.0015000163269089928,
     'Bytes written': 1048576,
     'Files written': 20816,
     'Events written': 1,
     'Bytes written total': 17319880101925.0,
     'Bytes written subrun': 1048576,
     'Bytes written uncompressed': 17422558}}}},
 'System': {'Clients': {'2049309': {'Run state': 0},
   '2049311': {'Run state': 0}}},
 'Runinfo': {'Stop time': 'Fri Feb 21 15:18:30 2025',
  'Stop time binary': '0x00000000',
  'Transition in progress': 1},
 'Equipment': {'EBlvl': {'Statistics': {'Events sent': 28}},
  'EBuilder': {'Statistics': {'Events sent': 143},
   'Special dump numbers': {'Last BORR': -1, 'Last BORTS': -1}},
  'L2Trigger': {'Statistics': {'Events sent': 33}},
  'readoutfe_SDU': {'Statistics': {'Events sent': 110}},
  'readoutveto03': {'Statistics': {'Events sent': 0}},
  'L2TriggerHistory': {'Statistics': {'Events sent': 5}},
  'readouthistory03': {

In [ ]:
# json2 = filter_json_remove_slash_key(json2)
# json2 = filter_json_remove_system_keys(json2)
# json2 = filter_json_remove_legacytrigger(json2)
# json2 =filter_remove_last_written(json2)
# json2 = filter_json_differ_Readback_Setting(json2)

In [20]:
#Not Disk space its memory not relevant
# json2 = read_json_from_file('savedata1.json')
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# # Print size in MB with 3 decimal points
# print(f"Size of json2 : {size_in_mb:.2f} KB")

# json2 = filter_json_remove_slash_key(json2)
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove /Key : {size_in_mb:.2f} KB")

# json2 = filter_json_remove_system_keys(json2)
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove system expect client: {size_in_mb:.2f} KB")

# json2 = filter_json_remove_legacytrigger(json2)
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove legacytrigger: {size_in_mb:.2f} KB")

# json2 =json_filter_remove_last_written(json2)
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove last written: {size_in_mb:.2f} KB")

# # either way is same remove  not needed
# json2 = filter_json_differ_Readback_Setting(json2)
# size_in_bytes_comprehensive = asizeof.asizeof(json2)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove settings if Readback is same as Detectors/DetXX: {size_in_mb:.2f} KB")

# # either way is same remove 
# size_in_bytes_comprehensive = asizeof.asizeof(finddifference3)
# size_in_mb = size_in_bytes_comprehensive / (1024)
# print(f"Size of json2 remove if start is same as stop: {size_in_mb:.2f} KB")



In [30]:
json2 = read_json_from_file('24250227_135126_end_original.json')
json2 = filter_json_remove_slash_key_value(json2)
save_json_to_file(json2, "24250227_135126_end_remove_slash_key_value.json")
json2 = filter_json_remove_system_keys(json2)
save_json_to_file(json2, "24250227_135126_end_remove_system_keys_except_clients.json")

json2 = filter_json_remove_legacytrigger(json2)
save_json_to_file(json2, "24250227_135126_end_remove_legacytrigger.json")

save_json_to_file(finddifference, "24250227_135126_diff_start_end_ODB.json")

print("saving Json" )

saving Json


In [42]:
json2 = read_json_from_file('24250227_135126_start_original.json')
json2 = filter_json_remove_slash_key_value(json2)
save_json_to_file(json2, "24250227_135126_start_remove_slash_key_value.json")
json2 = filter_json_remove_system_keys(json2)
save_json_to_file(json2, "24250227_135126_start_remove_system_keys_except_clients.json")

json2 = filter_json_remove_legacytrigger(json2)
save_json_to_file(json2, "24250227_135126_start_remove_legacytrigger.json")
save_json_to_file(finddifference, "24250227_135126_diff_end_withstart_ODB.json")
print("saving start Json" )

saving start Json


In [43]:
import os

# Define file names corresponding to each step with sizes in KB
file_sizes_on_disk = {
    "Original_json": 11785291/1024,
    "Json_removing_slash_key_values": 7795067 / 1024,
    "Json_removing_system_keys_except_clients": 7475285 / 1024,
    "Json_removing_legacytrigger": 7475285 / 1024,
    "Json_difference_in_end_ODB_with_start": 7955/1024
}
print()
# Print table header with better formatting
print(f"{'Action (odb end json for series=24250227_135126)':<50}{'Disk Size (KB)':>20}")
print("=" * 75)

# Print each file's disk size in a neatly formatted way
for file_name, size_kb in file_sizes_on_disk.items():
    print(f"{file_name:<50}{size_kb:>20.2f}")
print()
print()
print()


Action (odb end json for series=24250227_135126)        Disk Size (KB)
Original_json                                                 11509.07
Json_removing_slash_key_values                                 7612.37
Json_removing_system_keys_except_clients                       7300.08
Json_removing_legacytrigger                                    7300.08
Json_difference_in_end_ODB_with_start                             7.77





In [44]:
# Define file names corresponding to each step with sizes in KB
file_sizes_on_disk = {
    "Original_json": 4335590/1024,
    "Json_removing_slash_key_values": 7795110 / 1024,
    "Json_removing_system_keys_except_clients": 7475284 / 1024,
    "Json_removing_legacytrigger": 7475284 / 1024,
    
}
print()
# Print table header with better formatting
print(f"{'Action (odb start json for series=24250227_135126)':<50}{'Disk Size (KB)':>20}")
print("=" * 75)

# Print each file's disk size in a neatly formatted way
for file_name, size_kb in file_sizes_on_disk.items():
    print(f"{file_name:<50}{size_kb:>20.2f}")
print()
print()
print()


Action (odb start json for series=24250227_135126)      Disk Size (KB)
Original_json                                                  4233.97
Json_removing_slash_key_values                                 7612.41
Json_removing_system_keys_except_clients                       7300.08
Json_removing_legacytrigger                                    7300.08





In [19]:
import json

def read_json_with_error_handling(file_path):
    try:
        with open(file_path, 'r', encoding="utf-8") as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"JSON error at line {e.lineno}, column {e.colno}: {e.msg}")
        return None  # Or handle it as needed
read_json_with_error_handling('database_newend.json')

JSON error at line 1, column 4266656: Expecting ',' delimiter


In [27]:
import json

def save_json_compact(data, filename):
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, separators=(",", ":"))  # Compact format (no extra spaces)

# Load, filter, and save the compact file
def clean_json_file(input_file, output_file, pattern="/key"):
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)  

    filtered_data = filter_json_remove_slash_key_value(data, pattern)  

    save_json_compact(filtered_data, output_file)  # Save without spaces

# Example Usage
clean_json_file("abscend.json", "abscend_new.json")
